In [10]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\Carolina Mendoza\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [8]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [3]:
import requests
import pandas as pd
import json
import pymongo

In [4]:
#Set up API
#API_KEY = 'demo'
r = requests.get('https://api.twelvedata.com/time_series?symbol=AAPL,EUR/USD,IXIC&interval=1day&apikey=demo&source=docs')
result = r.json()

In [5]:
print (type(result).__name__)
print(result)

dict
{'AAPL': {'meta': {'symbol': 'AAPL', 'interval': '1day', 'currency': 'USD', 'exchange_timezone': 'America/New_York', 'exchange': 'NASDAQ', 'type': 'Common Stock'}, 'values': [{'datetime': '2022-04-28', 'open': '159.34000', 'high': '162.50999', 'low': '158.92999', 'close': '161.01500', 'volume': '53917943'}, {'datetime': '2022-04-27', 'open': '155.91000', 'high': '159.78999', 'low': '155.38000', 'close': '156.57001', 'volume': '87889100'}, {'datetime': '2022-04-26', 'open': '162.25000', 'high': '162.34000', 'low': '156.72000', 'close': '156.80000', 'volume': '95623200'}, {'datetime': '2022-04-25', 'open': '161.12000', 'high': '163.17000', 'low': '158.46001', 'close': '162.88000', 'volume': '96046400'}, {'datetime': '2022-04-22', 'open': '166.46001', 'high': '167.87000', 'low': '161.50000', 'close': '161.78999', 'volume': '84775200'}, {'datetime': '2022-04-21', 'open': '168.91000', 'high': '171.53000', 'low': '165.91000', 'close': '166.42000', 'volume': '87227800'}, {'datetime': '20

In [6]:
#Extract data from JSON
aapl = result['AAPL']
aapl_price=aapl['values']
print(aapl_price)

[{'datetime': '2022-04-28', 'open': '159.34000', 'high': '162.50999', 'low': '158.92999', 'close': '161.01500', 'volume': '53917943'}, {'datetime': '2022-04-27', 'open': '155.91000', 'high': '159.78999', 'low': '155.38000', 'close': '156.57001', 'volume': '87889100'}, {'datetime': '2022-04-26', 'open': '162.25000', 'high': '162.34000', 'low': '156.72000', 'close': '156.80000', 'volume': '95623200'}, {'datetime': '2022-04-25', 'open': '161.12000', 'high': '163.17000', 'low': '158.46001', 'close': '162.88000', 'volume': '96046400'}, {'datetime': '2022-04-22', 'open': '166.46001', 'high': '167.87000', 'low': '161.50000', 'close': '161.78999', 'volume': '84775200'}, {'datetime': '2022-04-21', 'open': '168.91000', 'high': '171.53000', 'low': '165.91000', 'close': '166.42000', 'volume': '87227800'}, {'datetime': '2022-04-20', 'open': '168.75999', 'high': '168.88000', 'low': '166.10001', 'close': '167.23000', 'volume': '67929800'}, {'datetime': '2022-04-19', 'open': '165.02000', 'high': '167.

In [7]:
#Transform data into Pandas Dataset
pt=pd.DataFrame(aapl_price)
print (type(pt).__name__)
print(pt)

DataFrame
      datetime       open       high        low      close     volume
0   2022-04-28  159.34000  162.50999  158.92999  161.01500   53917943
1   2022-04-27  155.91000  159.78999  155.38000  156.57001   87889100
2   2022-04-26  162.25000  162.34000  156.72000  156.80000   95623200
3   2022-04-25  161.12000  163.17000  158.46001  162.88000   96046400
4   2022-04-22  166.46001  167.87000  161.50000  161.78999   84775200
5   2022-04-21  168.91000  171.53000  165.91000  166.42000   87227800
6   2022-04-20  168.75999  168.88000  166.10001  167.23000   67929800
7   2022-04-19  165.02000  167.82001  163.91000  167.39999   67723800
8   2022-04-18  163.92000  166.60001  163.57001  165.07001   69023900
9   2022-04-14  170.62000  171.27000  165.03999  165.28999   75237500
10  2022-04-13  167.39000  171.03999  166.77000  170.39999   70618900
11  2022-04-12  168.02000  169.87000  166.64000  167.66000   79265200
12  2022-04-11  168.71001  169.03000  165.50000  165.75000   72246700
13  2022-0

In [33]:
#Set up Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

In [9]:
# Create dataset from Pandas Dataframe
sparkDF=spark.createDataFrame(pt) 
#Print dataset schema
sparkDF.printSchema()
sparkDF.show()

root
 |-- datetime: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)

+----------+---------+---------+---------+---------+---------+
|  datetime|     open|     high|      low|    close|   volume|
+----------+---------+---------+---------+---------+---------+
|2022-04-28|159.34000|162.50999|158.92999|161.01500| 53917943|
|2022-04-27|155.91000|159.78999|155.38000|156.57001| 87889100|
|2022-04-26|162.25000|162.34000|156.72000|156.80000| 95623200|
|2022-04-25|161.12000|163.17000|158.46001|162.88000| 96046400|
|2022-04-22|166.46001|167.87000|161.50000|161.78999| 84775200|
|2022-04-21|168.91000|171.53000|165.91000|166.42000| 87227800|
|2022-04-20|168.75999|168.88000|166.10001|167.23000| 67929800|
|2022-04-19|165.02000|167.82001|163.91000|167.39999| 67723800|
|2022-04-18|163.92000|166.60001|163.57001|165.07001| 69023900|
|2022-04-14|170.

In [63]:
#Get max close price 

from pyspark.sql.functions import max


q1=sparkDF.select(max ("close"))
q1.show()


+----------+
|max(close)|
+----------+
| 178.96001|
+----------+



In [69]:
#Get record for max close price

from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
windowDept = Window.orderBy(col("close").desc())
maxdate=sparkDF.withColumn("row",row_number().over(windowDept)) \
  .filter(col("row") == 1).drop("row") 
maxdate.show()
  

+----------+---------+---------+---------+---------+---------+
|  datetime|     open|     high|      low|    close|   volume|
+----------+---------+---------+---------+---------+---------+
|2022-03-29|176.69000|179.00999|176.34000|178.96001|100589400|
+----------+---------+---------+---------+---------+---------+



In [73]:
#Get date for max date
varmaxdate=maxdate.first()['datetime']
print(varmaxdate)
type(varmaxdate)


2022-03-29


str

In [61]:
#Get Minimum close price
from pyspark.sql.functions import min
q2=sparkDF.select(min ("close"))
q2.show()

+----------+
|min(close)|
+----------+
| 156.57001|
+----------+



In [74]:
#Get record for minimum close price
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
windowDept = Window.orderBy(col("close").asc())
mindate=sparkDF.withColumn("row",row_number().over(windowDept)) \
  .filter(col("row") == 1).drop("row") 
mindate.show()

+----------+---------+---------+---------+---------+--------+
|  datetime|     open|     high|      low|    close|  volume|
+----------+---------+---------+---------+---------+--------+
|2022-04-27|155.91000|159.78999|155.38000|156.57001|87889100|
+----------+---------+---------+---------+---------+--------+



In [75]:
#Get date min close price
varmindate=mindate.first()['datetime']
print(varmindate)
type(varmindate)

2022-04-27


str

In [12]:
#Set up NEWS API 

r_news = requests.get('https://eodhistoricaldata.com/api/news?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX&s=AAPL')
result_news = r_news.json()
print(result_news)

[{'date': '2022-04-28T13:56:00+00:00', 'title': 'Stock Market Today - 4/28: Stocks Get Earnings, Tech Boost; GDP Disappoints, Apple In Focus', 'content': "Tech earnings are set to give Wall Street an early boost Thursday, with Apple's Q2 update, GDP data and a surging U.S. dollar also in focus.", 'link': 'https://finance.yahoo.com/m/6df62194-41d6-317a-9e54-b107a0a784d4/stock-market-today-4%2F28%3A.html', 'symbols': ['AAPL.US', 'AAPL34.SA', 'AAPL.MX', 'APC.F', 'APC.XETRA'], 'tags': ['APPLE', 'QUARTER EARNINGS', 'CONSUMER SENTIMENT'], 'sentiment': {'polarity': 0.402, 'neg': 0, 'neu': 0.899, 'pos': 0.101}}, {'date': '2022-04-28T13:33:02+00:00', 'title': 'Dow Jones Rallies As Facebook Soars 18% On Earnings; Apple Earnings Due', 'content': 'The Dow Jones Industrial Average jumped Thursday, as Facebook soared 18% on earnings results. Apple earnings are due after the market closes.', 'link': 'https://finance.yahoo.com/m/7e485e19-3054-333c-9147-2f1e0024f996/dow-jones-rallies-as-facebook.html',

In [9]:
#Set up News API

from pyspark.sql import SparkSession, functions as F
from urllib.request import urlopen

url = 'https://eodhistoricaldata.com/api/news?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX&s=AAPL'
jsonData = urlopen(url).read().decode('utf-8')
rdd = spark.sparkContext.parallelize([jsonData])
df = spark.read.json(rdd)

AttributeError: module 'pyspark' has no attribute 'sparkContext'

In [ ]:
#Print News schema
df.printSchema()
df.show()

root
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- link: string (nullable = true)
 |-- sentiment: struct (nullable = true)
 |    |-- neg: double (nullable = true)
 |    |-- neu: double (nullable = true)
 |    |-- polarity: double (nullable = true)
 |    |-- pos: double (nullable = true)
 |-- symbols: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             content|                date|                link|           sentiment|             symbols|                tags|               title|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Dow Jones futures...|20

In [17]:
#Transform date into date format
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, IntegerType

df=df.withColumn("date",df["date"].cast(DateType()))

df.printSchema()
df.show()

root
 |-- content: string (nullable = true)
 |-- date: date (nullable = true)
 |-- link: string (nullable = true)
 |-- sentiment: struct (nullable = true)
 |    |-- neg: double (nullable = true)
 |    |-- neu: double (nullable = true)
 |    |-- polarity: double (nullable = true)
 |    |-- pos: double (nullable = true)
 |-- symbols: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             content|      date|                link|           sentiment|             symbols|                tags|               title|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Tech earnings are...|2022-04-28|https://finance.y...|{0

Volume of articles by Date 

In [18]:
#Count number of articles by date
df.groupBy("date").count().show()



+----------+-----+
|      date|count|
+----------+-----+
|2022-04-27|   31|
|2022-04-28|   19|
+----------+-----+



Articles by date that contain the word Apple on the title

In [25]:
#df.filter(col("title").contains("Apple")).count()
df.groupBy(col("date")).agg(count(when(col("title").contains("Apple"),1))).show()




+----------+--------------------------------------------------+
|      date|count(CASE WHEN contains(title, Apple) THEN 1 END)|
+----------+--------------------------------------------------+
|2022-04-27|                                                13|
|2022-04-28|                                                10|
+----------+--------------------------------------------------+



In [26]:
#Query: search articles that content the word Apple
df.groupBy(col("date")).agg(count(when(col("content").contains("Apple"),1))).show()

+----------+----------------------------------------------------+
|      date|count(CASE WHEN contains(content, Apple) THEN 1 END)|
+----------+----------------------------------------------------+
|2022-04-27|                                                  21|
|2022-04-28|                                                  14|
+----------+----------------------------------------------------+



Flattening Sentiment Analysis data 

In [35]:
#Unnesting Sentiment Analysis from dataset
df_sentiment= df.select(['content','date','link','sentiment.neg', 'sentiment.neu','sentiment.polarity','sentiment.pos', 'symbols', 'tags', 'title'])
df_sentiment.show()


+--------------------+----------+--------------------+-----+-----+--------+-----+--------------------+--------------------+--------------------+
|             content|      date|                link|  neg|  neu|polarity|  pos|             symbols|                tags|               title|
+--------------------+----------+--------------------+-----+-----+--------+-----+--------------------+--------------------+--------------------+
|Tech earnings are...|2022-04-28|https://finance.y...|  0.0|0.899|   0.402|0.101|[AAPL.US, AAPL34....|[APPLE, QUARTER E...|Stock Market Toda...|
|The Dow Jones Ind...|2022-04-28|https://finance.y...|  0.0|  1.0|     0.0|  0.0|[TSLA.US, AAPL.US...|[DOW JONES, STOCK...|Dow Jones Rallies...|
|In this article, ...|2022-04-28|https://finance.y...|0.045|0.838|   0.999|0.116|[AAPL.US, AMZN.US...|[MICROSOFT CORPOR...|10 Stocks You Sho...|
|(For a Reuters li...|2022-04-28|https://finance.y...|0.071|0.851|  -0.664|0.079|[AAPL.US, FB.US, ...|[FACEBOOK, NASDAQ...|US STOC

In [37]:
df_sentiment.select('date','neg','neu','pos').show()

+----------+-----+-----+-----+
|      date|  neg|  neu|  pos|
+----------+-----+-----+-----+
|2022-04-28|  0.0|0.899|0.101|
|2022-04-28|  0.0|  1.0|  0.0|
|2022-04-28|0.045|0.838|0.116|
|2022-04-28|0.071|0.851|0.079|
|2022-04-28|0.171|0.829|  0.0|
|2022-04-28|  0.0|  1.0|  0.0|
|2022-04-28|  0.1|0.826|0.074|
|2022-04-28|0.102|0.808| 0.09|
|2022-04-28| 0.07|0.811|0.119|
|2022-04-28|  0.0|0.844|0.156|
|2022-04-28| 0.18|0.683|0.137|
|2022-04-28| 0.11|0.797|0.093|
|2022-04-28|0.052|0.834|0.115|
|2022-04-28|0.062|0.832|0.106|
|2022-04-28|  0.0|0.909|0.091|
|2022-04-28|0.091|0.909|  0.0|
|2022-04-28|0.089|0.795|0.116|
|2022-04-28|0.077|0.826|0.097|
|2022-04-28|  0.0|  1.0|  0.0|
|2022-04-27|0.065|0.853|0.082|
+----------+-----+-----+-----+
only showing top 20 rows



In [41]:
#Sentiment Analysis average per date 
df_sentiment.groupBy("date") \
    .avg("neg","neu","polarity","pos") \
    .show()


+----------+-------------------+------------------+-------------------+-------------------+
|      date|           avg(neg)|          avg(neu)|      avg(polarity)|           avg(pos)|
+----------+-------------------+------------------+-------------------+-------------------+
|2022-04-27|0.03780645161290323|0.8711612903225805| 0.5194516129032258|0.09100000000000001|
|2022-04-28|0.06421052631578945|0.8574210526315792|0.18642105263157896|0.07842105263157896|
+----------+-------------------+------------------+-------------------+-------------------+

